##  All figures

This code is largely based on the implementation provided by Enge et al. (2021), available at https://osf.io/34ry2/. We are deeply grateful for their dedication to open research.

> Enge, A., Abdel Rahman, R., & Skeide, M. A. (2021). A meta-analysis of fMRI studies of semantic cognition in children. NeuroImage, 241, 118436. https://doi.org/10.1016/j.neuroimage.2021.118436

In [1]:
# Import necessary modules
from os import makedirs, path
from atlasreader import get_statmap_info
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
from nilearn import image, plotting, reporting
from scipy import stats
from scipy.stats import pearsonr
import os, fnmatch
from glob import glob
from pathlib import Path
import re

The Python package you are importing, AtlasReader, is licensed under the
BSD-3 license; however, the atlases it uses are separately licensed under more
restrictive frameworks.
By using AtlasReader, you agree to abide by the license terms of the
individual atlases. Information on these terms can be found online at:
https://github.com/miykael/atlasreader/tree/master/atlasreader/data



In [15]:
def combined_cluster_table(
    img_files_z=[],
    img_files_ale=[],
    stub_keys=[],
    stub_colname="Analysis",
    cluster_extent= 200,
    atlas="harvard_oxford",
    td_jar=None,
    output_file="",
):

    # Create output director
    output_dir = path.dirname(output_file)
    makedirs(output_dir, exist_ok=True)

    # Create a list of DataFrames with peak and cluster stats for each image
    df_tuples = [
        get_statmap_info(img_file, cluster_extent=cluster_extent, atlas="harvard_oxford", voxel_thresh=0)
        for img_file in img_files_z
    ]
    dfs = [
        pd.DataFrame(
            {
                "Cluster #": df_tuple[0]["cluster_id"],
                "Size (mm3)": df_tuple[0]["volume_mm"],
                "Cluster labels": df_tuple[0][atlas],
                "Mean z": df_tuple[0]["cluster_mean"],
                "Peak z": df_tuple[1]["peak_value"],
                "Peak X": df_tuple[1]["peak_x"],
                "Peak Y": df_tuple[1]["peak_y"],
                "Peak Z": df_tuple[1]["peak_z"],
                "Peak label": df_tuple[1][atlas],
            }
        )
        for df_tuple in df_tuples
    ]

    # Add ALE values if available
    if img_files_ale:
        df_tuples_ale = [
            get_statmap_info(img_file, cluster_extent=0, atlas="harvard_oxford", voxel_thresh=0)
            if img_file
            else (
                pd.DataFrame({"cluster_mean": [float("nan")]}),
                pd.DataFrame({"peak_value": [float("nan")]}),
            )
            for img_file in img_files_ale
        ]
        dfs_ale = [
            pd.DataFrame(
                {
                    "Mean ALE": df_tuple[0]["cluster_mean"],
                    "Peak ALE": df_tuple[1]["peak_value"],
                }
            )
            for df_tuple in df_tuples_ale
        ]
        for df, df_ale in zip(dfs, dfs_ale):
            df.insert(4, column="Mean ALE", value=df_ale["Mean ALE"])
            df.insert(6, column="Peak ALE", value=df_ale["Peak ALE"])

    # Concatenate into one big DataFrame
    df = pd.concat(dfs, keys=stub_keys)

    # Reformat numerical columns
    df["Size (mm3)"] = df["Size (mm3)"].apply(lambda x: "{:,.0f}".format(x))
    cols_int = ["Cluster #", "Peak X", "Peak Y", "Peak Z"]
    df[cols_int] = df[cols_int].applymap(int)
    cols_2f = ["Mean z", "Peak z"]
    df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
    if img_files_ale:
        cols_3f = ["Mean ALE", "Peak ALE"]
        df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))
        df[cols_3f] = df[cols_3f].replace("nan", "")

    # Add the stub column
    df.index = df.index.set_names(stub_colname, level=0)
    df.reset_index(level=stub_colname, inplace=True)
    mask = df[stub_colname].duplicated()
    df.loc[mask.values, [stub_colname]] = ""

    # Save to CSV
    df.to_csv(output_file, sep="\t", index=False)

    return df

### Tables - ALE results

#### Table 1.  Single analysis: healthy control group

In [21]:
# Table 1 -  single analysis - healthy control group

ale_thresh_z_control = "../results/ale/control_all_z_size_level_thresh.nii.gz"
ale_thresh_ale_control = "../results/ale/control_all_stat_size_thresh.nii.gz"
output_file = "../results/tables/tab_ale_control.tsv"

tab1 = combined_cluster_table(
    img_files_z=[ale_thresh_z_control],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=[ale_thresh_ale_control, None],
    atlas="harvard_oxford",
    output_file=output_file
)

display(tab1)


/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_57213/3152266981.py:68: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_int] = df[cols_int].applymap(int)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_57213/3152266981.py:70: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_57213/3152266981.py:73: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))


,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"15,672",26.39% Left_Paracingulate_Gyrus; 18.43% Left_C...,3.72,0.022,3.72,0.049,0,48,8,37.0% Left_Paracingulate_Gyrus; 20.0% Right_Pa...
1,,2,"4,080",88.04% Left_Frontal_Orbital_Cortex; 6.27% Left...,3.72,0.021,3.72,0.042,-38,24,-14,63.0% Left_Frontal_Orbital_Cortex
2,,3,"3,288",43.80% Left_Cingulate_Gyrus_posterior_division...,3.72,0.023,3.72,0.043,-2,-54,26,67.0% Left_Cingulate_Gyrus_posterior_division;...
3,,4,"1,216",34.87% Left_Cingulate_Gyrus_posterior_division...,2.62,0.022,2.62,0.036,2,-16,38,59.0% Right_Cingulate_Gyrus_posterior_division...
4,,5,"1,096",90.51% Left_Angular_Gyrus; 9.49% Left_Lateral_...,2.37,0.022,2.37,0.035,-44,-58,24,61.0% Left_Angular_Gyrus; 17.0% Left_Lateral_O...
5,,6,968,71.90% Right_Frontal_Orbital_Cortex; 21.49% Ri...,2.06,0.020,2.06,0.029,32,18,-18,71.0% Right_Frontal_Orbital_Cortex; 8.0% Right...


#### Table 2.  Single analysis: psychiatric group

In [22]:
# Table 2 -  single analysis - psychiatric group

ale_thresh_z_patients = "../results/ale/patient_z_size_level_thresh.nii.gz"
ale_thresh_ale_patients = "../results/ale/patient_stat_size_thresh.nii.gz"
output_file = "../results/tables/tab_ale_patients.tsv"

tab2 = combined_cluster_table(
    img_files_z=[ale_thresh_z_patients],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=[ale_thresh_ale_patients, None],
    atlas="harvard_oxford",
    output_file=output_file
)

display(tab2)

/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_57213/3152266981.py:68: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_int] = df[cols_int].applymap(int)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_57213/3152266981.py:70: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_57213/3152266981.py:73: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))


,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"1,376",79.07% Left_Frontal_Pole; 11.63% Left_Paracing...,3.09,0.010,3.09,0.014,-6,58,18,50.0% Left_Frontal_Pole; 11.0% Left_Superior_F...
